# Khai báo thư viện

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

# Đọc dữ liệu (giả định file CSV không có header, thêm header theo pima-indians-diabetes.names)

In [ ]:
column_names = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 
                'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
data = pd.read_csv('pima-indians-diabetes.csv', names=column_names)

# 1. Xác định giá trị thiếu và mẫu bất thường

In [ ]:
print("=== Xác định giá trị thiếu ===")
missing_values = data.isnull().sum()
print("Số lượng giá trị thiếu cho mỗi cột:")
print(missing_values)

# Kiểm tra giá trị 0 (có thể là giá trị thiếu theo ngữ cảnh y tế, ví dụ: Glucose=0 không hợp lý)

In [ ]:
print("\n=== Kiểm tra giá trị 0 (có thể là giá trị thiếu) ===")
zero_values = (data == 0).sum()
print("Số lượng giá trị 0 cho mỗi cột:")
print(zero_values)

# Xác định mẫu bất thường (dựa trên các giá trị không hợp lý, ví dụ: Glucose=0, BloodPressure=0)

In [ ]:
print("\n=== Mẫu bất thường (giá trị 0 không hợp lý) ===")
unreasonable_zeros = data[(data['Glucose'] == 0) | (data['BloodPressure'] == 0) | 
                         (data['BMI'] == 0) | (data['Insulin'] == 0)]
print("Số lượng mẫu có giá trị 0 bất hợp lý:")
print(unreasonable_zeros.shape[0])
print(unreasonable_zeros.head())

# 2. Xác định ngoại lệ

In [ ]:
print("\n=== Xác định ngoại lệ ===")

# Vẽ boxplot cho từng cột số để phát hiện ngoại lệ

In [ ]:
plt.figure(figsize=(12, 8))
data.boxplot(column=column_names[:-1])  # Loại bỏ cột Outcome vì là nhãn
plt.title("Boxplot của các biến để phát hiện ngoại lệ")
plt.xticks(rotation=45)
plt.show()

# Tính IQR để xác định ngoại lệ chi tiết

In [ ]:
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

# Kiểm tra ngoại lệ cho từng cột

In [ ]:
for column in column_names[:-1]:  # Loại bỏ cột Outcome
    outliers = detect_outliers(data, column)
    print(f"\nSố lượng ngoại lệ trong cột {column}: {outliers.shape[0]}")
    print(outliers[[column]].head())

# Vẽ heatmap để trực quan hóa tương quan (giúp phát hiện bất thường gián tiếp)

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(data.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Ma trận tương quan giữa các biến")
plt.show()

# Kết thúc